In [17]:
import visa
import time
import numpy as np
import pymeasure
from pymeasure.instruments.keithley import Keithley2400
import pandas as pd
import matplotlib.pyplot as plt
import datetime

import subprocess
def get_position():
    result = subprocess.check_output([r'C:\Program Files\IronPython 2.7\ipy.exe', 
                                      r'C:\Users\yanglabuser\Desktop\Sam files\_switch\ppms-master\ppms_position.py'])

    current_p = round(float(result.decode('utf-8').strip().split(',')[1][:-1]))
    return current_p

def set_T(t=300):
    if t < 2 or t > 400:
        raise
    result = subprocess.check_output([r'C:\Program Files\IronPython 2.7\ipy.exe', 
                                      r'C:\Users\yanglabuser\Desktop\Sam files\_switch\ppms-master\ppms.py', str(t)])

    current_t = result.decode('utf-8').strip()
    if abs(float(current_t)-t)<1:
        print("current T {}".format(current_t))
        return
    else:
        print("set t:")
        print(t)
        print("current t:")
        print(current_t)
        raise




def set_Field(field=0, persistent=False):
    if abs(field) > 140000:
        raise
    if persistent:
        result = subprocess.check_output([r'C:\Program Files\IronPython 2.7\ipy.exe', 
                                      r'C:\Users\yanglabuser\Desktop\Sam files\_switch\ppms-master\ppms_set_field.py',
                                          str(field), str('persistent')])
    else:
        result = subprocess.check_output([r'C:\Program Files\IronPython 2.7\ipy.exe', 
                                          r'C:\Users\yanglabuser\Desktop\Sam files\_switch\ppms-master\ppms_set_field.py', 
                                          str(field)])

    current_field = result.decode('utf-8').strip()
    print(current_field)
    if abs(float(current_field)-field)<2:
        print("current field {}".format(current_field))
        return
    else:
        print("set field:")
        print(field)
        print("current t:")
        print(current_field)
        raise
        
sensitivity = {1:0, 0.1:3, 0.01:6, 0.001:9, 0.5:1, 0.00005:13,
               0.0001:12, 0.00001:15, 0.000001:18, 0.0002:11, 0.05:4, 0.00002:14}

def set_position(p):
    if p >360 and p < -10:
        raise
    subprocess.check_output([r'C:\Program Files\IronPython 2.7\ipy.exe', 
                                      r'C:\Users\yanglabuser\Desktop\Sam files\_switch\ppms-master\ppms_move_p.py',
                                          str(p)])

class Lockin:
    def __init__(self, resource, gpib=4):
        self.lock_in = resource.open_resource('GPIB0::{}::INSTR'.format(gpib))

    def change_harmonic(self, harm=1, sens=1):
        self.lock_in.write("HARM {}".format(harm))
        self.lock_in.write("SCAL {}".format(sensitivity[sens]))

    def lock_in_read(self):
        x = float(self.lock_in.query("OUTP? 0").strip())
        y = float(self.lock_in.query("OUTP? 1").strip())
        r = float(self.lock_in.query("OUTP? 2").strip())
        theta = float(self.lock_in.query("OUTP? 3").strip())
        return x, y, r, theta

    def lock_in_measure(self, count=10, time_step=0.1,
                        wait_before_measure=3,
                        harm=2, sens=0.00001):
        self.change_harmonic(harm, sens)
        time.sleep(wait_before_measure)
        xs, ys, rs, thetas = [], [], [], []
        for i in range(count):
            x, y, r, theta = self.lock_in_read()
            xs += [x]
            ys += [y]
            rs += [r]
            thetas += [theta]
            time.sleep(time_step)
        x_mean, x_std = np.mean(xs), np.std(xs)
        y_mean, y_std = np.mean(ys), np.std(ys)
        r_mean, r_std = np.mean(rs), np.std(rs)
        theta_mean, theta_std = np.mean(thetas), np.std(thetas)

        return x_mean, y_mean, r_mean, theta_mean, \
               x_std, y_std,r_std, theta_std

#Check 6221 compliance, Check 2400 output off state

In [18]:
import datetime
print(datetime.datetime.now())

2020-02-22 15:37:29.648620


In [19]:
import os

In [20]:
print(os.getcwd())

C:\Users\Admin\Desktop\Sam files\_switch


In [21]:
rm = visa.ResourceManager()
rm.list_resources()

('ASRL3::INSTR', 'ASRL10::INSTR', 'GPIB0::10::INSTR')

In [22]:
lock = Lockin(rm, 10)

In [23]:
lock.lock_in_read()

(0.0070050731301, 8.9864988695e-05, 0.0070312051103, 1.234572053)

In [24]:
get_position()

0

In [25]:
def lock_angular_measurement_set_p(positions, lock, first_range=0.01, second_range=0.00001, time_step=0.1,
                             input_I =0.001, wait_before_measure=30, 
                             lock_count=10, sample_id="Fe2O3",  
                             time_after_move=5, time_out_count = 60*5):
    filename = "./data/" + sample_id + "_{}.csv".format(int(time.time()))
    with open(filename, 'w') as f:
        f.write("x,y,r,theta,std_x,std_y,std_r,std_theta,x2,y2,r2,theta2,std_x2,std_y2,std_r2,std_theta2,ac_I,POS\n")
    
    for pos in positions:
        set_position(pos)
        time.sleep(time_after_move)
        print("current position: {}".format(pos))
        res1 = lock.lock_in_measure(lock_count, time_step, wait_before_measure, harm=1,sens=first_range)
        res2 = lock.lock_in_measure(lock_count, time_step, wait_before_measure, harm=2,sens=second_range)

        with open(filename, 'a') as f:
            f.write("{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{}\n".format(*res1, *res2, input_I, pos))
    set_position(0)
    time.sleep(180)

In [34]:
do_task = True

In [35]:
ps = list(range(-5, 15, 3)) + list(range(15, 165, 10)) + list(range(165, 195, 3)) + list(range(195, 346, 10))

In [19]:
#ps = list(range(-5, 346, 10))

In [20]:
#ps = list(range(0,361,10))

In [36]:
len(ps)

48

In [39]:
do_task = True

In [40]:
#rxy measurment 14, 10
fields = [10000, 20000, 60000, 100000]
temps = [300, 10]
#set_T(temp)
#time.sleep(25*60)
if not do_task:
    raise
for temp in temps:
    for f in fields:
        set_Field(f, True)
        set_T(temp)
        time.sleep(5*60)
        lock_angular_measurement_set_p(ps, lock, sample_id="2nmPt_Fe2o3_30nm_test_1p5mA_rxy_gamma"+ str(temp) + "K" + str(f)+ "G", 
                                       first_range=0.1, second_range=0.00001,wait_before_measure=60)
     
set_Field(0, True)
set_T(300)
do_task=False

10000.2246094
current field 10000.2246094
current T 299.9977
current position: -5
current position: -2
current position: 1
current position: 4
current position: 7
current position: 10
current position: 13
current position: 15
current position: 25
current position: 35
current position: 45
current position: 55
current position: 65
current position: 75
current position: 85
current position: 95
current position: 105
current position: 115
current position: 125
current position: 135
current position: 145
current position: 155
current position: 165
current position: 168
current position: 171
current position: 174
current position: 177
current position: 180
current position: 183
current position: 186
current position: 189
current position: 192
current position: 195
current position: 205
current position: 215
current position: 225
current position: 235
current position: 245
current position: 255
current position: 265
current position: 275
current position: 285
current position: 295
current posi

current position: 165
current position: 168
current position: 171
current position: 174
current position: 177
current position: 180
current position: 183
current position: 186
current position: 189
current position: 192
current position: 195
current position: 205
current position: 215
current position: 225
current position: 235
current position: 245
current position: 255
current position: 265
current position: 275
current position: 285
current position: 295
current position: 305
current position: 315
current position: 325
current position: 335
current position: 345
-0.0158353056759
current field -0.0158353056759
current T 300.0469
